In [57]:
import requests
from bs4 import BeautifulSoup
import mysql.connector

In [58]:
from datetime import datetime
import concurrent.futures

# List of days to check
days_to_check = [1, 2, 3, 15, 16, 17, 30, 31]

# List of months to check
months_to_check = range(1, 13)

# Range of years to check
year_start = 2006
year_end = 2025

# Generate all specified dates in the range of years
available_dates = []

def check_date(year, month, day):
	try:
		date = datetime(year + 543, month, day)
		test_url = f"https://news.sanook.com/lotto/check/{date.strftime('%d%m')}{date.strftime('%Y')}/"
		response = requests.get(test_url)
		if response.status_code == 200:
			soup = BeautifulSoup(response.content, 'html.parser')
			div_r5 = soup.find('div', class_='lottocheck__sec lottocheck__sec--font-mini')
			if div_r5:
				numbers_r5 = [span.text for span in div_r5.find_all('span', class_='lotto__number')]
				if numbers_r5:
					available_dates.append(date.strftime("%Y-%m-%d"))
	except ValueError:
		# Skip invalid dates (e.g., April 31)
		pass

with concurrent.futures.ThreadPoolExecutor() as executor:
	futures = []
	for year in range(year_start, year_end + 1):
		for month in months_to_check:
			for day in days_to_check:
				futures.append(executor.submit(check_date, year, month, day))
	concurrent.futures.wait(futures)

print(available_dates)


['2549-12-30', '2550-01-16', '2550-02-16', '2550-03-01', '2550-03-16', '2550-02-01', '2550-04-01', '2550-04-16', '2550-05-02', '2550-05-16', '2550-06-01', '2550-06-16', '2550-07-16', '2550-07-01', '2550-08-16', '2550-08-01', '2550-09-01', '2550-09-16', '2550-10-01', '2550-10-16', '2550-11-01', '2550-11-16', '2550-12-16', '2550-12-30', '2551-02-01', '2551-01-16', '2551-03-01', '2551-02-16', '2551-03-16', '2551-04-01', '2551-04-16', '2551-05-16', '2551-05-02', '2551-06-01', '2551-06-16', '2551-07-01', '2551-07-16', '2551-08-01', '2551-08-16', '2551-09-01', '2551-10-01', '2551-09-16', '2551-10-16', '2551-11-01', '2551-12-01', '2551-11-16', '2551-12-16', '2551-12-30', '2552-01-16', '2552-02-01', '2552-02-16', '2552-03-01', '2552-03-16', '2552-04-01', '2552-04-16', '2552-05-02', '2552-05-16', '2552-06-01', '2552-06-16', '2552-07-16', '2552-07-01', '2552-08-01', '2552-09-01', '2552-08-16', '2552-09-16', '2552-10-01', '2552-10-16', '2552-11-16', '2552-12-01', '2552-11-01', '2552-12-30', '2552

In [59]:
from dotenv import load_dotenv
import os
# Connect database
def establish_connection():
    load_dotenv()
    return mysql.connector.connect(
        host=os.getenv('DB_HOST'),
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASSWORD'),
        database=os.getenv('DB_NAME')
    )

In [64]:
import pandas as pd
from datetime import datetime
import mysql.connector
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os

# Connect to the MySQL database using environment variables
conn = establish_connection()
cursor = conn.cursor()

# Define the select and insert queries
select_query = "SELECT COUNT(*) FROM lottery WHERE date = %s AND reward_type = %s"
insert_query = "INSERT INTO lottery (date, reward_type, number) VALUES (%s, %s, %s)"

# Function to convert Buddhist year to Gregorian year
def convert_buddhist_to_gregorian(date_str):
    buddhist_date = datetime.strptime(date_str, '%Y-%m-%d')
    gregorian_year = buddhist_date.year - 543
    gregorian_date = buddhist_date.replace(year=gregorian_year)
    return gregorian_date.strftime('%Y-%m-%d')

# Function to process and insert lottery numbers
def process_lottery_numbers(date_str, numbers, reward_type):
    gregorian_date_str = convert_buddhist_to_gregorian(date_str)
    cursor.execute(select_query, (gregorian_date_str, reward_type))
    result = cursor.fetchone()
    if result[0] == 0:
        for number in numbers:
            cursor.execute(insert_query, (gregorian_date_str, reward_type, number))
    else:
        print(f"Date {gregorian_date_str} with reward type {reward_type} already exists in the database")
        
def extract_numbers(soup, reward_text):
    divs = soup.find_all('div', class_='lottocheck__sec')
    numbers = []
    for div in divs:
        if reward_text in div.text:
            numbers.extend([span.text for span in div.find_all('span', class_='lotto__number')])
    return numbers

# Use a session to reuse the connection
with requests.Session() as session:
    for date_str in available_dates:
        buddhist_date = datetime.strptime(date_str, '%Y-%m-%d')
        test_url = f"https://news.sanook.com/lotto/check/{buddhist_date.strftime('%d%m')}{buddhist_date.strftime('%Y')}/"
        response = session.get(test_url)
        print(f'Current Date is {date_str} | URL is {test_url}')
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            numbers_r2 = extract_numbers(soup, "รางวัลที่ 2 มี 5 รางวัลๆละ 200,000 บาท") or extract_numbers(soup, "รางวัลที่ 2 มี 5 รางวัลๆละ 100,000 บาท")
            numbers_r3 = extract_numbers(soup, "รางวัลที่ 3 มี 10 รางวัลๆละ 80,000 บาท") or extract_numbers(soup, "รางวัลที่ 3 มี 10 รางวัลๆละ 40,000 บาท")
            numbers_r4 = extract_numbers(soup, "รางวัลที่ 4 มี 50 รางวัลๆละ 40,000 บาท") or extract_numbers(soup, "รางวัลที่ 4 มี 50 รางวัลๆละ 20,000 บาท")
            numbers_r5 = extract_numbers(soup, "รางวัลที่ 5 มี 100 รางวัลๆละ 20,000 บาท") or extract_numbers(soup, "รางวัลที่ 5 มี 100 รางวัลๆละ 10,000 บาท")
            
            process_lottery_numbers(date_str, numbers_r2, "2")
            process_lottery_numbers(date_str, numbers_r3, "3")
            process_lottery_numbers(date_str, numbers_r4, "4")
            process_lottery_numbers(date_str, numbers_r5, "5")
            print(numbers_r2)
        else:
            print(f"Failed to fetch URL for date: {date_str}")

# Commit the transaction once at the end
conn.commit()

# Close the database connection
conn.close()

print("Dates have been updated to Gregorian format in the database and saved to lottery_data.xlsx")

Current Date is 2549-12-30 | URL is https://news.sanook.com/lotto/check/30122549/
Date 2006-12-30 with reward type 2 already exists in the database
Date 2006-12-30 with reward type 4 already exists in the database
Date 2006-12-30 with reward type 5 already exists in the database
['133134', '468424', '509756', '515202', '640052']
Current Date is 2550-01-16 | URL is https://news.sanook.com/lotto/check/16012550/
Date 2007-01-16 with reward type 2 already exists in the database
Date 2007-01-16 with reward type 4 already exists in the database
Date 2007-01-16 with reward type 5 already exists in the database
['004887', '218489', '373187', '716200', '801651']
Current Date is 2550-02-16 | URL is https://news.sanook.com/lotto/check/16022550/
Date 2007-02-16 with reward type 2 already exists in the database
Date 2007-02-16 with reward type 4 already exists in the database
Date 2007-02-16 with reward type 5 already exists in the database
['051060', '317491', '344177', '569792', '814070']
Current

In [65]:
# test_url = "https://news.sanook.com/lotto/check/02012568/"
# soup = BeautifulSoup(requests.get(test_url).content, 'html.parser')
# print(soup.prettify())
# def extract_numbers(soup, reward_text):
#     divs = soup.find_all('div', class_='lottocheck__sec')
#     numbers = []
#     for div in divs:
#         if reward_text in div.text:
#             numbers.extend([span.text for span in div.find_all(
#                 'span', class_='lotto__number')])
#     return numbers

# # Extract numbers for reward type 3 and 4
# numbers_r2 = extract_numbers(soup, "รางวัลที่ 2 มี 5 รางวัลๆละ 200,000 บาท")
# numbers_r3 = extract_numbers(soup, "รางวัลที่ 3 มี 10 รางวัลๆละ 80,000 บาท")
# numbers_r4 = extract_numbers(soup, "รางวัลที่ 4 มี 50 รางวัลๆละ 40,000 บาท")
# numbers_r5 = extract_numbers(soup, "รางวัลที่ 5 มี 100 รางวัลๆละ 20,000 บาท")

# print("Numbers for reward type 3:", numbers_r3)
# print("Numbers for reward type 4:", numbers_r4)
# print("Numbers for reward type 5:", numbers_r5)

In [66]:
import mysql.connector
import pandas as pd

# Re-establish the connection to the MySQL database
conn = establish_connection()

# Define the reward types
reward_types = ['2', '3', '4', '5']

dfs = {}
for reward_type in reward_types:
    query = f"SELECT * FROM lottery WHERE reward_type = '{reward_type}'"
    dfs[reward_type] = pd.read_sql(query, conn)

# Create a new column 'date' to store the date in datetime format
# Convert the 'date' column to datetime format
for reward_type, df in dfs.items():
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

# Verify the conversion
print(dfs['2']['date'].head())

# Ensure the date format is yyyy-mm-dd when exporting to Excel
for reward_type, df in dfs.items():
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')

# Save the DataFrame to an Excel file
for reward_type, df in dfs.items():
    df.to_excel(f"lottery_data_type{reward_type}.xlsx", index=False)
    
df_all = pd.concat(dfs.values(), ignore_index=True)
df_all.to_excel("lottery_data.xlsx", index=False)

print("Data has been saved to lottery_data_typeX.xlsx")

# Close the database connection
conn.close()

C:\Users\Kolawat\AppData\Local\Temp\ipykernel_22156\417736900.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfs[reward_type] = pd.read_sql(query, conn)


0   2006-12-30
1   2006-12-30
2   2006-12-30
3   2006-12-30
4   2006-12-30
Name: date, dtype: datetime64[ns]
Data has been saved to lottery_data_typeX.xlsx
